## Modelo 3 - Resultado do Active learning

Aqui vamos medir o resultado do active learning. 

In [1]:
#importando pacotes
import pandas as pd
import numpy as np
import re
import time

import bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 100)

#https://strftime.org
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
#lendo dataframes
df_limpo = pd.read_pickle('./datasets/df_limpo.pkl')

In [4]:
df_limpo.head()

,data,views,title
0,2019-07-30,635,Finanças é Coisa de Criança!
1,2019-12-03,37305,Câncer 2020 - Profissional e Finanças 1° semestre
2,2017-03-30,98,Seja Rica: Conquiste sua Independência Financeira
3,2019-03-19,41,Independência FINANCEIRA
5,2020-01-21,389,6 ERROS para NÃO ter a INDEPENDÊNCIA FINANCEIR...


In [5]:
features = pd.read_pickle('./datasets/features.pkl')

In [6]:
features.head()

,views,views_por_dia
0,635,2.243816
1,37305,237.611465
2,98,0.086344
3,41,0.098558
5,389,3.601852


In [2]:
#carregando conjunto completo de dados, somente com as anotações feitas na primeira fase
df1 = pd.read_excel('./datasets/raw_data_with_labels.xlsx', index_col=0)
df1 = df1[df1.y.notnull()].copy()
df1.shape

(503, 14)

In [6]:
#carregando conjunto de dados do active learning
df2 = pd.read_excel('./datasets/active_label.xlsx', index_col=0)
df2 = df2[df2.y.notnull()].copy()
df2['novo'] = 1
df2.shape

(100, 17)

In [7]:
df2.head(2)

,watch-title,y,watch-view-count,watch-time-text,watch7-headline,watch8-sentiment-actions,og:image,og:image:width,og:description,og:video:width,og:video:height,og:video:tag,content_watch-info-tag-list,channel_link_0,data,p,novo
566,TOP 10 AÇÕES PARA INVESTIR EM MAIO DE 2020 - ...,0,772 visualizações,Publicado em 2 de mai. de 2020,TOP 10 AÇÕES PARA INVESTIR EM MAIO DE 2020 - ...,772 visualizações\n\n\n\n\n\n\n\n27\n\nGostou ...,https://i.ytimg.com/vi/M8MSZipIWyg/maxresdefau...,1280,TOP 10 AÇÕES PARA INVESTIR EM MAIO DE 2020 - A...,1280,720,COMO INVESTIR,Pessoas e blogs,/channel/UCL6FUaPa_l8MAzrk1oqxCFw,2020-05-02,0.548,1
579,Qual é o melhor Investimento? Comprar imóveis ...,1,2.555 visualizações,Estreou em 5 de mai. de 2020,Qual é o melhor Investimento? Comprar imóveis ...,2.555 visualizações\n\n\n\n\n\n\n\n454\n\nGost...,https://i.ytimg.com/vi/MhUjrYvJXrw/maxresdefau...,1280,Separei outras Dicas dentro do E-book de FII's...,1280,720,passiva,Licença de atribuição Creative Commons (reutil...,/channel/UCSXdP8V4jRaw-8YMTpa6glw,2020-05-05,0.514,1


1ª checagem: Quais seriam os valores das métricas no dataset recém etiquetado (df2) utilizando as "probabilidades" do primeiro modelo? Aqui, não há parâmetro de comparação sobre a qualidade do resultado, mas sim sobre a proximidade com os valores esperados.

In [8]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [9]:
average_precision_score(df2.y, df2.p), roc_auc_score(df2.y, df2.p)

(0.3801329540333924, 0.600873161764706)

Os valores para essas métricas nos dados de validação foram, respectivamente:
* 0.466503359462486
* 0.8083843329253366

Aqui, a pergunta a fazer é: Essa diferença é pequena o suficiente para podermos afirmar que o modelo está generalizando as suas predições?


## Limpando dados novamente
Agora, será criado um novo dataframe a partir da junção de df1 e df2, retirando a coluna *p*. Como a coluna *novo* existe somente no df2, ela será criada nos dados provenientes do df1 com valores NaN.

In [10]:
df = pd.concat([df1, df2.drop('p', axis=1)])

In [11]:
#criando dataframe com os índices do df
df_limpo = pd.DataFrame(index = df.index)
df_limpo['title'] = df['watch-title']
df_limpo['novo'] = df['novo'].fillna(0) #valores nan (que vieram do df1) vão virar 0

In [14]:
map_mes = {'jan.':'Jan',
             'fev.':'Feb',
              'mar.':'Mar',
              'abr.':'Apr',
              'mai.':'May',
              'jun.':'Jun',
              'jul.':'Jul',
              'ago.':'Aug',
              'set.':'Sep',
              'out.':'Oct',
              'nov.':'Nov',
              'dez.':'Dec'}

In [13]:
def limpa_data (row):
    
    data_limpa = re.search(r"(\d+) de ([a-z]+)\. de (\d+)", row.loc['watch-time-text']).group()
    data_limpa = data_limpa.split('de ')
    data_limpa = [x.strip() for x in data_limpa]
    data_limpa[0] = ['0' + str(data_limpa[0]) if len(data_limpa[0]) == 1 else data_limpa[0]][0]
    data_limpa[1] = [v for k, v in map_mes.items() if k == data_limpa[1]][0]
    data = '-'.join(data_limpa)

    return data

In [15]:
#aplicando procedimento de limpeza da data
df_limpo['data'] = df.apply(limpa_data, axis=1)

In [16]:
df_limpo['data'] = pd.to_datetime(df_limpo['data'])

In [17]:
#adicionando coluna com view count formatado
df_limpo['views'] = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".","").fillna(0).astype(int)

In [20]:
df_limpo.sample(5)

,title,novo,data,views
765,Escolha Enriquecer: Guia Completo da Independê...,1.0,2020-04-08,104
30,Conselhos sobre Finanças e Familia,0.0,2011-12-02,15081
149,Aplicando árvore de decisão no Ibovespa - Fina...,0.0,2018-03-02,3758
268,"🚨 10 AÇÕES (BIDI4, GOLL4, CVCB3, OIBR3, PETR4,...",0.0,2020-03-30,44739
426,Escolha Enriquecer: Guia Completo da Independê...,0.0,2020-04-22,26


## Criando features

In [23]:
features = pd.DataFrame(index=df_limpo.index)
y = df.y.copy()

In [24]:
#criando coluna com dias que se passaram desde a publicação até a data referência
features['dias_publicado'] = (pd.to_datetime("2020-05-08") - df_limpo['data']) / np.timedelta64(1, 'D') 
#denominador cria um objeto timedelta do numpy em diferença de 1 dia
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views']/features['dias_publicado']
features = features.drop('dias_publicado', axis = 1)

In [25]:
features.head()

,views,views_por_dia
0,635,2.243816
1,37305,237.611465
2,98,0.086344
3,41,0.098558
5,389,3.601852


## Avaliando efeito do Active Learning

### 1º caso: Treino com dados sem active learning e validação com o dataset aumentado (com os dados etiquetados antes e depois)

In [26]:
#importando alguns modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [27]:
#criando sets de treino e validação, separando os dados em 50% (quantil 0.5)
mask_train = (df_limpo.data < df_limpo.data.quantile(0.5)) & (df_limpo['novo'] == 0)

mask_val = df_limpo.data >= df_limpo.data.quantile(0.5)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]

In [28]:
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((255, 2), (302, 2), (255,), (302,))

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title'] #coletando títulos do set de treino
title_val = df_limpo[mask_val]['title'] #coletando títulos do set de validação

title_vec = TfidfVectorizer(min_df = 2) #vetorizador. lembrar que modelos não trabalham com strings
#ocorrência mínima do termo em 2 vídeos

title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #usamos somente transform para transformar somente baseado nas palavras no dataset passado para ele.

In [30]:
title_bow_train.shape  

(255, 214)

In [31]:
from scipy.sparse import hstack, vstack

In [32]:
Xtrain_wtitle = hstack([Xtrain, title_bow_train]) #aqui, juntamos as features de treino em Xtrain com as geradas para os títulos das features de treino
Xval_wtitle = hstack([Xval, title_bow_val]) #ídem para validação

In [33]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((255, 216), (302, 216))

In [34]:
%%time
#treinando modelo
# 1000 árvores

mdl = RandomForestClassifier(n_estimators = 1000, random_state = 0, class_weight="balanced", n_jobs=8)
mdl.fit(Xtrain_wtitle, ytrain)

Wall time: 1.89 s


RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=8,
                       random_state=0)

In [35]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [36]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [37]:
# resultado com validação aumentada
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.6584134705818345, 0.8734939759036144)

Valores anteriores eram:
* 0.7411156726976864
* 0.9096695226438188
- mindf = 2

Aqui vemos uma diminuição das métricas, que a princípio não significa muita coisa, devido ao tamanho da amostra de dados.

### 2º caso: Treino com dados aumentados e validação o dataset inicial (sem os dados etiquetados antes e depois)

In [39]:
#criando sets de treino e validação, separando os dados em 50% (quantil 0.5)
mask_train = (df_limpo.data < df_limpo.data.quantile(0.5)) 
mask_val = (df_limpo.data >= df_limpo.data.quantile(0.5)) & (df_limpo['novo'] == 0)

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]

title_train = df_limpo[mask_train]['title'] #coletando títulos do set de treino
title_val = df_limpo[mask_val]['title'] #coletando títulos do set de validação
title_vec = TfidfVectorizer(min_df = 2) #vetorizador. lembrar que modelos não trabalham com strings
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #usamos s

Xtrain_wtitle = hstack([Xtrain, title_bow_train]) 
Xval_wtitle = hstack([Xval, title_bow_val])

mdl = RandomForestClassifier(n_estimators = 1000, random_state = 0, class_weight="balanced", n_jobs=8)
mdl.fit(Xtrain_wtitle, ytrain)


p = mdl.predict_proba(Xval_wtitle)[:, 1]

#com treino aumentado
average_precision_score(yval, p), roc_auc_score(yval, p)


(0.7296859901459777, 0.9039323683873447)

mais uma vez, os valores "referência" eram:
    
* 0.7411156726976864
* 0.9096695226438188
- mindf = 2

Aqui, houve uma melhora de ambas as métricas em relação ao 1º caso.

### 3º caso: Treino e validação com os dados aumentados

In [40]:
#criando sets de treino e validação, separando os dados em 50% (quantil 0.5)
mask_train = (df_limpo.data < df_limpo.data.quantile(0.5)) 
mask_val = (df_limpo.data >= df_limpo.data.quantile(0.5))

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]

title_train = df_limpo[mask_train]['title'] #coletando títulos do set de treino
title_val = df_limpo[mask_val]['title'] #coletando títulos do set de validação
title_vec = TfidfVectorizer(min_df = 2) #vetorizador. lembrar que modelos não trabalham com strings
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #usamos s

Xtrain_wtitle = hstack([Xtrain, title_bow_train]) 
Xval_wtitle = hstack([Xval, title_bow_val])

mdl = RandomForestClassifier(n_estimators = 1000, random_state = 0, class_weight="balanced", n_jobs=8)
mdl.fit(Xtrain_wtitle, ytrain)


p = mdl.predict_proba(Xval_wtitle)[:, 1]

#com treino e validação aumentados
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.6624786239605276, 0.8764491929984087)

Aqui, vemos uma piora nas métricas. Juntando todos os casos, temos:

ANTES:  
ap, auc  
0.7411156726976864, 0.9096695226438188  
  
TREINO ANTIGO, VALIDAÇÃO AUMENTADA  
ap, auc  
0.6584134705818345, 0.8734939759036144  
  
TREINO AUMENTADO, VALIDAÇÃO ANTIGA  
ap, auc  
0.7296859901459777, 0.9039323683873447  
  
TREINO E VALIDAÇÃO AUMENTADOS  
ap, auc  
0.6624786239605276, 0.8764491929984087  


Conforme o próprio Mário coloca, não cabe comparar as métricas diretamente por que ela são geradas por validações e modelos com dados diferentes. A técnica serve para otimizar a escolha de dados para aumentar um dataset de trabalho, com base no active learning, e melhorar o desempenho de um dado modelo.

O próximo passo será de modelagem dos dados. Para isso, o resto das linhas no arquivo ./datasets/raw_data_with_labels.xlsx serão anotadas e salvas no novo arquivo ./datasets/raw_data_all_labels.xlsx.